# This notebook is for preparing data which will be the input of the system

All of the data is from [OurWorldInData.org](https://ourworldindata.org/)  

**IMPORTANT NOTES**  
1. Years to include: the year that the policy was published and 10 years preceding.
2. The data will mostly focus to the specific country but the average world and region data will also be included as well.

## Setup
First, get required packages and set environment variables.

In [1]:
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv

load_dotenv()

import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

Next, we need to define Neo4j credentials and connection. Follow these [installation steps](https://neo4j.com/docs/operations-manual/current/installation/) to set up a Neo4j database.

In [2]:
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Neo4jGraph()

## Load Data

Load csv file, this is the file we are going to add the inputs in.

In [3]:
import pandas as pd

policy_df = pd.read_csv('csv/test.csv')
policy_df.head()

,index,Document ID,Document Title,Family ID,Family Title,Family Summary,Collection Title(s),Collection Description(s),Document Role,Document Variant,...,Internal Collection ID(s),have_support_doc?,raw_doc_path,doc_id,text_file_path,lang,alpha-3,region,sub-region,intermediate-region
0,4162,agriculture-development-strategy-2015-2035_3d89,Agriculture Development Strategy 2015-2035,agriculture-development-strategy-2015-2035_9b59,Agriculture Development Strategy 2015-2035,The Agriculture Development Strategy (ADS) pro...,NaN,NaN,Main,NaN,...,NaN,True,climate_laws/1_pdfs/document_4080.pdf,document_4080,../climate_laws\en_cleaned_text_2\document_408...,en,NPL,Asia,Southern Asia,NaN
1,0,act-on-the-promotion-of-the-development-use-an...,"Act on the Promotion of the Development, Use a...",act-on-the-promotion-of-the-development-use-an...,"Act on the Promotion of the Development, Use a...",The revised Act on the Promotion of the Develo...,NaN,NaN,Main,Original Language,...,NaN,True,climate_laws/1_pdfs/document_0.pdf,document_0,../climate_laws\en_cleaned_text_1\document_0.txt,en,KOR,Asia,Eastern Asia,NaN
2,6488,the-national-strategy-for-biofuel-development-...,"The National Strategy for Biofuel Development,...",the-national-strategy-for-biofuel-development-...,"The National Strategy for Biofuel Development,...",This Strategy examines how renewable energy so...,NaN,NaN,Main,NaN,...,NaN,True,climate_laws/1_pdfs/document_6313.pdf,document_6313,../climate_laws\en_cleaned_text_2\document_631...,en,MLI,Africa,Sub-Saharan Africa,Western Africa
3,4744,agriculture-sector-plan-2015-2019_16c6,Agriculture Sector Plan 2015-2019,agriculture-sector-plan-2015-2019_7999,Agriculture Sector Plan 2015-2019,The Agriculture Sector Plan aims at improving ...,NaN,NaN,Main,NaN,...,NaN,True,climate_laws/1_pdfs/document_4644.pdf,document_4644,../climate_laws\en_cleaned_text_2\document_464...,en,NIU,Oceania,Polynesia,NaN
4,4663,vision-2030-the-chad-we-want_0349,"Vision 2030, The Chad We Want",vision-2030-the-chad-we-want_ceb0,"Vision 2030, The Chad We Want",This document provides the strategic framework...,NaN,NaN,Main,NaN,...,NaN,True,climate_laws/1_pdfs/document_4566.pdf,document_4566,../climate_laws\en_cleaned_text_2\document_456...,en,TCD,Africa,Sub-Saharan Africa,Middle Africa


## Climate data

Since our task is to find the related climate policy and most of the policies focus on improving the climate at their specific areas or global state. Therefore, climate data is important for policymakers when designing a new policy.

In [4]:
from owid import catalog
import pandas as pd

# Keywords for filtering climate-related data
c_keywords = ['co2', 'ghg', 'emission', 'diaster', 'forest', 'climate', 'greenhouse', 'pm2']
c_namespaces = ['climate', 'climate_watch', 'forests']

# Tables to exclude based on irrelevant categories
exclude_keywords = ['cigarette', 'tobacco', 'transport', 'children' , 'owid']

climate_data = []

# Loop to create a list of dataframe of climate data
for nsp in c_namespaces:
    df = catalog.find(namespace=nsp)
    # Filter rows where 'dimensions' contains both 'country' and 'year'
    filtered_df = df[df['dimensions'].apply(lambda x: ('country' in x or 'location' in x) and 'year' in x)]
    climate_data.append(filtered_df)

for kw in c_keywords:
    df = catalog.find(kw)
    filtered_df = df[df['dimensions'].apply(lambda x: ('country' in x or 'location' in x) and 'year' in x)]
    climate_data.append(filtered_df)

# Concatenate dataframes into one and drop duplicates based on the 'table' column
climate_data_df = pd.concat(climate_data)
climate_data_df = climate_data_df.drop_duplicates(subset='table', keep='last')

# Exclude tables that contain irrelevant keywords
climate_data_df = climate_data_df[~climate_data_df['table'].str.contains('|'.join(exclude_keywords), case=False)]

print('Number of climate data tables: ', len(climate_data_df))

Number of climate data tables:  34


Save data

In [5]:
rc = catalog.RemoteCatalog()

climate_data_ids = []
for idx, row in climate_data_df.iterrows():
    df = rc[row['path']]

    # Reset the index to make 'country' and 'year' regular columns
    df = df.reset_index()

    df.to_csv(f'../owid/climate_data/{idx}.csv', index=False)
    
    climate_data_ids.append(idx)

## Socio-Economics data

In [6]:
from owid import catalog
import pandas as pd

# Namespaces related to socio-economics data
s_namespaces = ['artificial_intelligence', 'democracy', 'health', 'war', 'demography', 
                'biodiversity', 'happiness', 'education', 'worldbank_wdi', 
                'research_development', 'urbanization', 'tourism']

# Extended list of keywords to exclude irrelevant data
exclude_keywords = [
    'nuclear', 'gini', 'polio', 'weapons', 'plastic_surgery', 'transplantation', 
    'organ', 'peace', 'suicide', 'robots', 'ai_phds', 'ai_robots', 'semiconductors', 
    'surgery', 'cancer', 'mid_country', 'lexical_index', 'patents', 
    'articles', 'life_expectancy', 'education_barro_lee_projections',
    'iucn_animal', 'pollution', 'isaps', 'death', 'suicide', 'mental_health', 'terrorism',
    'plastic_emissions', 'climate_change_attributable', 'epoch', 'cow', 'chupilkin_koczan', 'country',
    'location', 'cherry', 'broken', 'species', 'countdown', 'gpei', 'cset', 'wgm', 'unaids',
    'life_tables', 'sex', 'doubling_times', 'survivor', 'gender', 'wrp', 'wdi', 'ai_jobs', 
    'vdem_multi_with_regions', 'vdem_multi_without_regions'
]

# List to collect dataframes from socio-economics tables
socio_economics_data = []

# Loop to create a list of dataframes for socio-economic data
for nsp in s_namespaces:
    df = catalog.find(namespace=nsp)
    # Filter rows where 'dimensions' contains both 'country' and 'year'
    filtered_df = df[df['dimensions'].apply(lambda x: ('country' in x or 'location' in x) and 'year' in x)]
    socio_economics_data.append(filtered_df)

# Concatenate dataframes into one and drop duplicates based on the 'table' column
socio_economics_data_df = pd.concat(socio_economics_data)
socio_economics_data_df = socio_economics_data_df.drop_duplicates(subset='table', keep='last')

# Exclude tables that contain irrelevant keywords
socio_economics_data_df = socio_economics_data_df[~socio_economics_data_df['table'].str.contains('|'.join(exclude_keywords), case=False)]

# Display the number of remaining socio-economics tables
print('Number of socio economics tables after extended filtering: ', len(socio_economics_data_df))

Number of socio economics tables after extended filtering:  52


Save data

In [7]:
rc = catalog.RemoteCatalog()

socio_economics_data_ids = []
for idx, row in socio_economics_data_df.iterrows():
    df = rc[row['path']]

    # Reset the index to make 'country' and 'year' regular columns
    df = df.reset_index()
    
    if idx in climate_data_ids:
        continue

    df.to_csv(f'../owid/socio_economics_data/{idx}.csv', index=False)

    socio_economics_data_ids.append(idx)

## Other Related data

In [8]:
# Namespaces related to policy data
o_keywords = ['transport', 'vehicle', 'land_use', 'buildings', 'water', 'waste', 'industrial', 'ocean']
o_namespaces = ['energy', 'agriculture']

# Extended list of keywords to exclude irrelevant data
exclude_keywords = ['uk', 'injuries', 'injury', 'smoking', 'sex', 'gender', 'diarrhoea', 'children']

# List to collect dataframes from socio-economics tables
data = []

# Loop to create a list of dataframes for other data
for nsp in o_namespaces:
    df = catalog.find(namespace=nsp)
    # Filter rows where 'dimensions' contains both 'country' and 'year'
    filtered_df = df[df['dimensions'].apply(lambda x: ('country' in x or 'location' in x) and 'year' in x)]
    data.append(filtered_df)
    
# Loop to create a list of dataframes for other data
for kw in o_keywords:
    df = catalog.find(kw)
    # Filter rows where 'dimensions' contains both 'country' and 'year'
    filtered_df = df[df['dimensions'].apply(lambda x: ('country' in x or 'location' in x) and 'year' in x)]
    data.append(filtered_df)

# Concatenate dataframes into one and drop duplicates based on the 'table' column
data_df = pd.concat(data)
data_df = data_df.drop_duplicates(subset='table', keep='last')

# Exclude tables that contain irrelevant keywords
data_df = data_df[~data_df['table'].str.contains('|'.join(exclude_keywords), case=False)]

# Display the number of remaining socio-economics tables
print('Number of others related tables after extended filtering: ', len(data_df))

Number of others related tables after extended filtering:  44


In [9]:
rc = catalog.RemoteCatalog()

data_ids = []
for idx, row in data_df.iterrows():
    df = rc[row['path']]

    # Reset the index to make 'country' and 'year' regular columns
    df = df.reset_index()
    
    if idx in climate_data_ids or idx in socio_economics_data_ids:
        continue

    df.to_csv(f'../owid/other_data/{idx}.csv', index=False)

    data_ids.append(idx)

## Let's put them together

In [10]:
policy_df['climate_data'] = str(climate_data_ids)
policy_df['socio_economics_data'] = str(socio_economics_data_ids)
policy_df['other_data'] = str(data_ids)

policy_df.to_csv('csv/test_agents_role.csv', index=False)

policy_df.head()

,index,Document ID,Document Title,Family ID,Family Title,Family Summary,Collection Title(s),Collection Description(s),Document Role,Document Variant,...,doc_id,text_file_path,lang,alpha-3,region,sub-region,intermediate-region,climate_data,socio_economics_data,other_data
0,4162,agriculture-development-strategy-2015-2035_3d89,Agriculture Development Strategy 2015-2035,agriculture-development-strategy-2015-2035_9b59,Agriculture Development Strategy 2015-2035,The Agriculture Development Strategy (ADS) pro...,NaN,NaN,Main,NaN,...,document_4080,../climate_laws\en_cleaned_text_2\document_408...,en,NPL,Asia,Southern Asia,NaN,"[10231, 11601, 7691, 15294, 1390, 1392, 1394, ...","[1768, 1771, 1781, 1784, 1790, 3126, 3329, 336...","[6177, 6267, 6914, 6918, 7194, 12949, 13239, 1..."
1,0,act-on-the-promotion-of-the-development-use-an...,"Act on the Promotion of the Development, Use a...",act-on-the-promotion-of-the-development-use-an...,"Act on the Promotion of the Development, Use a...",The revised Act on the Promotion of the Develo...,NaN,NaN,Main,Original Language,...,document_0,../climate_laws\en_cleaned_text_1\document_0.txt,en,KOR,Asia,Eastern Asia,NaN,"[10231, 11601, 7691, 15294, 1390, 1392, 1394, ...","[1768, 1771, 1781, 1784, 1790, 3126, 3329, 336...","[6177, 6267, 6914, 6918, 7194, 12949, 13239, 1..."
2,6488,the-national-strategy-for-biofuel-development-...,"The National Strategy for Biofuel Development,...",the-national-strategy-for-biofuel-development-...,"The National Strategy for Biofuel Development,...",This Strategy examines how renewable energy so...,NaN,NaN,Main,NaN,...,document_6313,../climate_laws\en_cleaned_text_2\document_631...,en,MLI,Africa,Sub-Saharan Africa,Western Africa,"[10231, 11601, 7691, 15294, 1390, 1392, 1394, ...","[1768, 1771, 1781, 1784, 1790, 3126, 3329, 336...","[6177, 6267, 6914, 6918, 7194, 12949, 13239, 1..."
3,4744,agriculture-sector-plan-2015-2019_16c6,Agriculture Sector Plan 2015-2019,agriculture-sector-plan-2015-2019_7999,Agriculture Sector Plan 2015-2019,The Agriculture Sector Plan aims at improving ...,NaN,NaN,Main,NaN,...,document_4644,../climate_laws\en_cleaned_text_2\document_464...,en,NIU,Oceania,Polynesia,NaN,"[10231, 11601, 7691, 15294, 1390, 1392, 1394, ...","[1768, 1771, 1781, 1784, 1790, 3126, 3329, 336...","[6177, 6267, 6914, 6918, 7194, 12949, 13239, 1..."
4,4663,vision-2030-the-chad-we-want_0349,"Vision 2030, The Chad We Want",vision-2030-the-chad-we-want_ceb0,"Vision 2030, The Chad We Want",This document provides the strategic framework...,NaN,NaN,Main,NaN,...,document_4566,../climate_laws\en_cleaned_text_2\document_456...,en,TCD,Africa,Sub-Saharan Africa,Middle Africa,"[10231, 11601, 7691, 15294, 1390, 1392, 1394, ...","[1768, 1771, 1781, 1784, 1790, 3126, 3329, 336...","[6177, 6267, 6914, 6918, 7194, 12949, 13239, 1..."


### Transform time series data to text

In [14]:
import os
import pandas as pd
import json

def transform_time_series_to_text(folder, file_names, year, country, region):
    result = ""

    for file_name in file_names:
        # Load the CSV file
        csv_path = os.path.join(folder, str(file_name)+'.csv')
        df = pd.read_csv(csv_path)

        # Check if 'country' and 'location' columns exist and filter accordingly
        conditions = []

        if 'country' in df.columns:
            conditions.append(df['country'] == region)
            if country == 'European Union':
                conditions.append(df['country'].str.contains('European Union', na=False))
            else:
                conditions.append(df['country'] == country)

        if 'location' in df.columns:
            conditions.append(df['location'] == 'World')
            conditions.append(df['location'] == region)

        # Combine conditions with OR (|)
        if conditions:
            combined_condition = conditions[0]
            for condition in conditions[1:]:
                combined_condition |= condition
            filtered_df = df[combined_condition]
        else:
            # Skip the file if neither 'country' nor 'location' columns exist
            continue

        # Filter by the time span (5 years before the given year and up to the given year)
        filtered_df = filtered_df[(filtered_df['year'] >= (year - 5)) & (filtered_df['year'] <= year)]

        # Skip the file if the filtered_df is empty
        if filtered_df.empty:
            continue
        
        # Load metadata file (assuming it's a .meta.json file with the same name as CSV)
        meta_file_path = os.path.join(folder, str(file_name)+'.meta.json')
        with open(meta_file_path, 'r') as meta_file:
            meta_data = json.load(meta_file)
        
        # Extract the title
        if meta_data.get('title', None):
            table_title = meta_data.get('title')
        else:
            table_title = meta_data.get('dataset').get('title')
        
        # Convert the dataframe to a string format
        table_string = '[START_OF_TABLE]\n' + table_title + '\n'
        table_string += filtered_df.to_csv(index=False, header=False, sep=',') + '[END_OF_TABLE]\n'
        
        # Append to the result string
        result += table_string
    
    return result

### Generate inputs

I want to check the cost of each query

In [18]:
import ast
from langchain import hub
from langchain_community.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI
from tqdm.notebook import tqdm

other_data = ""
climate_data = ""
socio_economic_data = ""

input_data = dict()

i = 0

for idx, row in tqdm(policy_df.iterrows(), total=len(policy_df), desc='Genearting Input query'):        
    i+= 1
    if i < 87:
        continue
        
    year = row['Last event in timeline'].split('-')[0]
    country = row['Geography']
    region = row['region']
    sectors = ', '.join(row['Sector'].split(';'))

    # setup result
    input_data = dict()

    # transform time series data to text format
    file_names = ast.literal_eval(row['other_data'])
    other_data = transform_time_series_to_text('../owid/other_data', file_names, int(year), country, region)

    file_names = ast.literal_eval(row['climate_data'])
    climate_data = transform_time_series_to_text('../owid/climate_data', file_names, int(year), country, region)

    file_names = ast.literal_eval(row['socio_economics_data'])
    socio_economics_data = transform_time_series_to_text('../owid/socio_economics_data', file_names, int(year), country, region)

    input_data['input_prompt'] = dict()
    input_data['output'] = dict()
    # do analysis with advisors
    llm = ChatOpenAI(model='gpt-4o', temperature=0.7)

    # climate data
    prompt = hub.pull("analyzer-climate-data")
    with get_openai_callback() as cb:
        # save input
        input_data['input_prompt']['climate_data'] = prompt.format(climate_data=climate_data, sectors=sectors)
        # run llm
        climate_result = llm.invoke(prompt.format(climate_data=climate_data, sectors=sectors))
        # save output
        input_data['output']['climate_data'] = dict()
        input_data['output']['climate_data']['content'] = climate_result.content
        input_data['output']['climate_data']['total_cost'] = cb.total_cost
        input_data['output']['climate_data']['total_tokens'] = cb.total_tokens
        input_data['output']['climate_data']['prompt_tokens'] = cb.prompt_tokens
        input_data['output']['climate_data']['completion_tokens'] = cb.completion_tokens

    # socio-economic data
    prompt = hub.pull("analyzer-socio-economic")
    with get_openai_callback() as cb:
        # save input
        input_data['input_prompt']['socio_economic_data'] = prompt.format(socio_economic_data=socio_economics_data, sectors=sectors)
        # run llm
        sce_result = llm.invoke(prompt.format(socio_economic_data=socio_economics_data, sectors=sectors))
        # save output
        input_data['output']['socio_economic_data'] = dict()
        input_data['output']['socio_economic_data']['content'] = sce_result.content
        input_data['output']['socio_economic_data']['total_cost'] = cb.total_cost
        input_data['output']['socio_economic_data']['total_tokens'] = cb.total_tokens
        input_data['output']['socio_economic_data']['prompt_tokens'] = cb.prompt_tokens
        input_data['output']['socio_economic_data']['completion_tokens'] = cb.completion_tokens

    # other data
    prompt = hub.pull("analyzer-other_data")
    with get_openai_callback() as cb:
        # save input
        input_data['input_prompt']['other_data'] = prompt.format(other_data=other_data, sectors=sectors)
        # run llm
        other_result = llm.invoke(prompt.format(other_data=other_data, sectors=sectors))
        # save output
        input_data['output']['other_data'] = dict()
        input_data['output']['other_data']['content'] = other_result.content
        input_data['output']['other_data']['total_cost'] = cb.total_cost
        input_data['output']['other_data']['total_tokens'] = cb.total_tokens
        input_data['output']['other_data']['prompt_tokens'] = cb.prompt_tokens
        input_data['output']['other_data']['completion_tokens'] = cb.completion_tokens

    # run policy analyst
    llm = ChatOpenAI(model='gpt-4o', temperature=0.4)
    prompt = hub.pull("policy-expert")
    with get_openai_callback() as cb:
        # save input
        input_data['input_prompt']['policy_expert'] = prompt.format(**{
            "climate_data": climate_result.content, "other_data": other_result.content, "socio_economic_data": sce_result.content,
            "country": country, "region": region, "sectors": sectors
        })
        # run llm
        expert_result = llm.invoke(prompt.format(**{
            "climate_data": climate_result.content, "other_data": other_result.content, "socio_economic_data": sce_result.content,
            "country": country, "region": region, "sectors": sectors
        }))
        # save output
        input_data['output']['policy_expert'] = dict()
        input_data['output']['policy_expert']['content'] = expert_result.content
        input_data['output']['policy_expert']['total_cost'] = cb.total_cost
        input_data['output']['policy_expert']['total_tokens'] = cb.total_tokens
        input_data['output']['policy_expert']['prompt_tokens'] = cb.prompt_tokens
        input_data['output']['policy_expert']['completion_tokens'] = cb.completion_tokens

    # run summarizer
    llm = ChatOpenAI(model='gpt-4o', temperature=0.7)
    prompt = hub.pull("summarizer-policy")
    with get_openai_callback() as cb:
        # save input
        input_data['input_prompt']['summarizer'] = prompt.format(**{'advisor_output': expert_result.content})
        # run llm
        summarizer_result = llm.invoke(prompt.format(**{'advisor_output': expert_result.content}))
        # save output
        input_data['output']['summarizer'] = dict()
        input_data['output']['summarizer']['content'] = summarizer_result.content
        input_data['output']['summarizer']['total_cost'] = cb.total_cost
        input_data['output']['summarizer']['total_tokens'] = cb.total_tokens
        input_data['output']['summarizer']['prompt_tokens'] = cb.prompt_tokens
        input_data['output']['summarizer']['completion_tokens'] = cb.completion_tokens

    # run summarizer
    llm = ChatOpenAI(model='gpt-4o', temperature=0.7)
    prompt = hub.pull("summarizer-policy")
    with get_openai_callback() as cb:
        # save input
        input_data['input_prompt']['summarizer_v1'] = prompt.format(**{'advisor_output': input_data['output']['policy_expert']})
        # run llm
        summarizer_result = llm.invoke(prompt.format(**{'advisor_output': input_data['output']['policy_expert']}))
        # save output
        input_data['output']['summarizer_v1'] = dict()
        input_data['output']['summarizer_v1']['content'] = summarizer_result.content
        input_data['output']['summarizer_v1']['total_cost'] = cb.total_cost
        input_data['output']['summarizer_v1']['total_tokens'] = cb.total_tokens
        input_data['output']['summarizer_v1']['prompt_tokens'] = cb.prompt_tokens
        input_data['output']['summarizer_v1']['completion_tokens'] = cb.completion_tokens

    # save json to file
    os.makedirs('output_data', exist_ok=True)
    with open(f'./output_data/{row["doc_id"]}.json', 'w') as file:
        json.dump(input_data, file)

print('DONE')

Genearting Input query:   0%|          | 0/100 [00:00<?, ?it/s]

DONE


In [19]:
import glob
import json

policy_df['summarizer_v1'] = None

# Loop through JSON files
for f in glob.glob('./output_data/*.json'):
    with open(f, 'r') as file:
        obj = json.load(file)

    # Extract document_id from the file name
    _, document_id = os.path.split(f)
    document_id = document_id.replace('.json', '')

    # Search for the document_id in policy_df and update the 'generated_summary' column
    policy_df.loc[policy_df['doc_id'] == document_id, 'summarizer_v1'] = obj['output']['summarizer_v1']['content']

In [20]:
policy_df.to_csv('csv/test_agents_role.csv', index=False)